In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Load the original spectrum data
spectrum_data = pd.read_csv('/path/to/original/spectrum.csv')

# Plot the original spectrum
plt.figure(figsize=(12, 6))
plt.plot(spectrum_data['EnergykeV'], spectrum_data['Counts'], label='Original Spectrum')
plt.xlabel('Energy (keV)')
plt.ylabel('Counts')
plt.title('Original Thorium Spectrum')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Load the background spectrum data
background_data = pd.read_csv('/path/to/background/spectrum.csv')

# Scale the background spectrum counts to match the original spectrum's time frame
background_data['ScaledCounts'] = background_data['Counts'] / 2

# Subtract the scaled background spectrum from the original spectrum
spectrum_data['CorrectedCounts'] = spectrum_data['Counts'] - background_data['ScaledCounts']

# Plot the corrected spectrum
plt.figure(figsize=(12, 6))
plt.plot(spectrum_data['EnergykeV'], spectrum_data['CorrectedCounts'], label='Background Corrected Spectrum')
plt.xlabel('Energy (keV)')
plt.ylabel('Counts')
plt.title('Corrected Thorium Spectrum')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

import numpy as np

# Define the number of bins
num_bins = 1024

# Bin the corrected counts
binned_counts, bin_edges = np.histogram(spectrum_data['EnergykeV'], bins=num_bins, weights=spectrum_data['CorrectedCounts'])

# Calculate the bin centers
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Plot the binned spectrum
plt.figure(figsize=(12, 6))
plt.plot(bin_centers, binned_counts, label='Binned Corrected Spectrum')
plt.xlabel('Energy (keV)')
plt.ylabel('Binned Corrected Counts')
plt.title('Binned Corrected Spectrum of Thorium Mantle (1024 Bins)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

from scipy.interpolate import CubicSpline

# Load efficiency curve data
efficiency_data = pd.read_csv('/path/to/efficiency.csv')

# Perform cubic spline interpolation
cs = CubicSpline(efficiency_data['Energy'], efficiency_data['Efficiency'])

# Interpolate over a fine grid
energy_interpolated = np.linspace(efficiency_data['Energy'].min(), efficiency_data['Energy'].max(), 1000)
efficiency_interpolated = cs(energy_interpolated)

# Plot the interpolated efficiency curve
plt.figure(figsize=(12, 6))
plt.plot(efficiency_data['Energy'], efficiency_data['Efficiency'], 'o', label='Efficiency Data Points')
plt.plot(energy_interpolated, efficiency_interpolated, label='Interpolated Efficiency', color='orange')
plt.xlabel('Energy (keV)')
plt.ylabel('Efficiency')
plt.title('Interpolated Efficiency Curve of Germanium Detector')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Cut the spectrum to match the energy range of the efficiency curve
trimmed_spectrum = spectrum_data[(spectrum_data['EnergykeV'] >= efficiency_data['Energy'].min()) &
                                 (spectrum_data['EnergykeV'] <= efficiency_data['Energy'].max())]

# Plot the trimmed spectrum
plt.figure(figsize=(12, 6))
plt.plot(trimmed_spectrum['EnergykeV'], trimmed_spectrum['CorrectedCounts'], label='Trimmed Spectrum')
plt.xlabel('Energy (keV)')
plt.ylabel('Counts')
plt.title('Trimmed Spectrum to Match Efficiency Curve Range')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Correct for efficiency to obtain real counts emitted by the source
real_counts = trimmed_spectrum['CorrectedCounts'] / efficiency_interpolated

# Sum total counts
total_counts = real_counts.sum()
print(f'Total counts in one whole day: {total_counts}')


In [ ]:
# Sum of the real counts to get the normalization factor
total_real_counts_sum = np.nansum(real_counts)

# Creating the probability distribution by dividing each count by the total counts
probability_distribution = real_counts / total_real_counts_sum

# Plotting the probability distribution function
plt.figure(figsize=(12, 6))
plt.plot(energy_bin_centers_in_range, probability_distribution, label='Probability Distribution', color='blue')
plt.xlabel('Energy (keV)')
plt.ylabel('Probability')
plt.title('Probability Distribution Function of the Real Counts Spectrum')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Convert energy from keV to MeV
energy_MeV = energy_bin_centers_in_range / 1000.0  # since 1 MeV = 1000 keV

# Prepare the data for the txt file
probability_data = pd.DataFrame({
    'Energy_MeV': energy_MeV,
    'Probability': probability_distribution
})

# Define the file path for the txt file
txt_file_path = '/mnt/data/probability_distribution.txt'

# Save the DataFrame to a txt file, with space as the delimiter
probability_data.to_csv(txt_file_path, sep=' ', index=False, header=False)

# Return the path for download
txt_file_path


In [ ]:
# Ensure that the floating-point numbers have exactly six digits after the decimal point
# by formatting the numbers as strings with six decimal places.

# Format the DataFrame with six decimal places
probability_data_formatted = probability_data.applymap(lambda x: f'{x:.6f}')

# Save the formatted DataFrame to a txt file, with space as the delimiter
txt_file_path_formatted = '/mnt/data/probability_distribution_formatted.txt'
with open(txt_file_path_formatted, 'w') as f:
    for index, row in probability_data_formatted.iterrows():
        f.write(f"{row['Energy_MeV']} {row['Probability']}\n")

# Return the path for download
txt_file_path_formatted
